# Study Workflow Automation #

## Project Setup ##

These cells involve the setup of a Study and automate steps in the workflow that can occur prior to subject enrollment.

**Pre-requisites**

1. git installed
2. Python >= 3.12

**Install dependencies: ipywdigets**

In [1]:
# Debug statements
!which python
!python --version

/Users/dmoreland/projects/cdisc-360i-notebooks/venv/bin/python
Python 3.13.7


In [2]:
# imports
import os
import urllib
from urllib.parse import urlparse
import ipywidgets as widgets
import subprocess

In [19]:
# Global variables
PROJECT_DIR = '/Users/dmoreland/projects/cdisc-360i-notebooks'
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
PROJECT_LIST = []

**Define functions to be used throughout study setup workflow**

In [4]:
# Define functions
def change_working_dir(working_dir):
    pwd = os.getcwd()

    if pwd == working_dir:
        print("INFO: Already in the {} project directory...".format(working_dir))
    else:
        print("INFO: Changing to the {} working directory...".format(working_dir))
        try:
            os.chdir(working_dir)
        except FileNotFoundError:
            print("ERROR: {} does not exist...".format(working_dir))

def git_clone(repo):
    project = urlparse(repo)[2]
    project_dir = project[project.rindex('/')+1: project.rindex('.')]
    print(project_dir)
    if project_dir:
        print("INFO: {} already exists.  Skipping clone...".format(project_dir))
    else:
        print("INFO: Cloning repository {}...".format(project_dir))
        !git clone repo
    return project_dir

## Upload and Validate the Study Design USDM JSON file ##

In [23]:
PROJECT_LIST.append(PROJECT_DIR)
print(PROJECT_LIST)

['/Users/dmoreland/projects/cdisc-360i-notebooks']


In [5]:
# Change to the directory into which to clone repo
change_working_dir(PROJECT_DIR)

print(os.getcwd())

# Clone repo
repo = 'https://github.com/pendingintent/cdisc-json-validation.git'
#working_dir = git_clone('https://github.com/pendingintent/cdisc-json-validation.git')
!git clone {repo}
project = urlparse(repo)[2]
working_dir = project[project.rindex('/')+1: project.rindex('.')]
print(working_dir)
change_working_dir(os.path.join(PROJECT_DIR, working_dir))
print(os.getcwd())

INFO: Already in the /Users/dmoreland/projects/cdisc-360i-notebooks project directory...
/Users/dmoreland/projects/cdisc-360i-notebooks
Cloning into 'cdisc-json-validation'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 85 (delta 39), reused 61 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 1.26 MiB | 17.26 MiB/s, done.
Resolving deltas: 100% (39/39), done.
cdisc-json-validation
INFO: Changing to the /Users/dmoreland/projects/cdisc-360i-notebooks/cdisc-json-validation working directory...
/Users/dmoreland/projects/cdisc-360i-notebooks/cdisc-json-validation


In [6]:
# Install project dependencies
!pip install -r requirements.txt --quiet

In [7]:
# Create a data directory, if required
print(DATA_DIR)
if os.path.exists(DATA_DIR):
    print("INFO: {} exists...".format(DATA_DIR))
else:
    print("INFO: Creating directory {}...".format(DATA_DIR))
    os.mkdir(DATA_DIR)

os.getcwd()

/Users/dmoreland/projects/cdisc-360i-notebooks/data
INFO: /Users/dmoreland/projects/cdisc-360i-notebooks/data exists...


'/Users/dmoreland/projects/cdisc-360i-notebooks/cdisc-json-validation'

In [8]:
# Verify correct current working directory
print(os.getcwd())

/Users/dmoreland/projects/cdisc-360i-notebooks/cdisc-json-validation


In [9]:
# Upload json file
# TODO: Replace mercury with ipywidgets FileUpload for simpler implmentation with less dependencies
#file = mr.File(label="File upload", max_file_size="10MB")
widgets.FileUpload(
    accept=".json",
    multiple=False
)

uploader = widgets.FileUpload()
display(uploader)

FileUpload(value=(), description='Upload')

In [12]:
uploaded_file = uploader.value[0]
uploaded_file_name = uploaded_file['name']
target_file = os.path.join(DATA_DIR, uploaded_file_name)
with open(target_file, "wb") as fp:
    fp.write(uploaded_file.content)


In [15]:
# Execute the validation script
#import subprocess

change_working_dir(os.path.join(PROJECT_DIR, working_dir))
result = subprocess.run(["python", "validate.py", target_file], capture_output=True, text=True)


if "Validation successful" not in result.stdout:
    print("ERROR: Validation of {} not successful...".format(target_file))
else:
    print("INFO: Validation of {} is successful...".format(target_file))
#print('stdout:', result.stdout)
#print('stderr:', result.stderr)

INFO: Already in the /Users/dmoreland/projects/cdisc-360i-notebooks/cdisc-json-validation project directory...
INFO: Validation of /Users/dmoreland/projects/cdisc-360i-notebooks/data/pilot_LLZT_protocol.json is successful...


**If validation of the USDM JSON file is successful, the file can be used in following downstream programs**

In [22]:
change_working_dir(PROJECT_DIR)
PROJECT_LIST = []

INFO: Already in the /Users/dmoreland/projects/cdisc-360i-notebooks project directory...
